In [1]:
import pandas as pd
import configparser
from glob import glob
config = configparser.ConfigParser()
import sys
config.read("../../env.ini")
data_home = config['DEFAULT']['data_home']
output_dir = config['DEFAULT']['output_dir']
local_lib = config['DEFAULT']['local_lib']
sys.path.append(local_lib)
from textparser import TextParser

OHCO = ['para_num', 'sent_num', 'token_num']
source_files = f'{data_home}/sotu'

In [2]:
LIB = pd.read_csv('LIB.csv', sep='|').set_index('speech_id')
LIB

,president,source_file_path,year,party
speech_id,,,,
1,washington,C:/Users/patso/Documents/DS5001//data/sotu\179...,1790,IND
2,washington,C:/Users/patso/Documents/DS5001//data/sotu\179...,1790,IND
3,washington,C:/Users/patso/Documents/DS5001//data/sotu\179...,1791,IND
4,washington,C:/Users/patso/Documents/DS5001//data/sotu\179...,1792,IND
5,washington,C:/Users/patso/Documents/DS5001//data/sotu\179...,1793,IND
...,...,...,...,...
110,trump,C:/Users/patso/Documents/DS5001//data/sotu\202...,2020,REP
111,biden,C:/Users/patso/Documents/DS5001//data/sotu\202...,2021,DEM
112,biden,C:/Users/patso/Documents/DS5001//data/sotu\202...,2022,DEM


In [3]:
clip_pats = [r"\*\*\*\s*START OF", r"\*\*\*\s*END OF"]

speeches = []
for speech in LIB.index:
    print("Tokenizing", LIB.loc[speech].president)
    src_file_path = LIB.loc[speech]['source_file_path']
    text = TextParser(src_file_path, ohco_pats=[('chap', r"\*\*\*\s*START OF",'m')],
                      clip_pats=clip_pats, use_nltk=True)
    text.verbose = False
    text.strip_hyphens = True
    text.strip_whitespace = True
    text.import_source().parse_tokens()
    text.TOKENS['speech_id'] = speech
    text.TOKENS = text.TOKENS.reset_index().set_index(['speech_id']+text.OHCO)
    speeches.append(text.TOKENS)
CORPUS = pd.concat(speeches).reset_index().drop(columns=['chap_id']).set_index(['speech_id']+OHCO).sort_index()

Tokenizing washington
Tokenizing washington
Tokenizing washington
Tokenizing washington
Tokenizing washington
Tokenizing washington
Tokenizing washington
Tokenizing washington
Tokenizing adams
Tokenizing adams
Tokenizing adams
Tokenizing adams
Tokenizing wilson
Tokenizing wilson
Tokenizing wilson
Tokenizing wilson
Tokenizing wilson
Tokenizing wilson
Tokenizing harding
Tokenizing harding
Tokenizing coolidge
Tokenizing roosevelt
Tokenizing roosevelt
Tokenizing roosevelt
Tokenizing roosevelt
Tokenizing roosevelt
Tokenizing roosevelt
Tokenizing roosevelt
Tokenizing roosevelt
Tokenizing roosevelt
Tokenizing roosevelt
Tokenizing roosevelt
Tokenizing truman
Tokenizing truman
Tokenizing truman
Tokenizing truman
Tokenizing truman
Tokenizing truman
Tokenizing eisenhower
Tokenizing eisenhower
Tokenizing eisenhower
Tokenizing eisenhower
Tokenizing eisenhower
Tokenizing eisenhower
Tokenizing eisenhower
Tokenizing eisenhower
Tokenizing kennedy
Tokenizing kennedy
Tokenizing kennedy
Tokenizing johnson

In [4]:
CORPUS

pos_tuple   pos token_str  \
speech_id para_num sent_num token_num                                     
1         0        0        0             (Fellow, NNP)   NNP    Fellow   
                            1          (Citizens, NNPS)  NNPS  Citizens   
                            2                  (of, IN)    IN        of   
                            3                 (the, DT)    DT       the   
                            4             (Senate, NNP)   NNP    Senate   
...                                                 ...   ...       ...   
114       267      1        1               (you,, PRP)   PRP      you,   
                            2              (thank, VBD)   VBD     thank   
                            3               (you,, PRP)   PRP      you,   
                            4               (thank, VB)    VB     thank   
                            5                (you., NN)    NN      you.   

                                       term_str  
speech_id para_num sent_num token_num            
1         0        0        0            fellow  
                            1          citizens  
                            2                of  
                            3               the  
                            4            senate  
...                                         ...  
114       267      1        1               you  
                            2             thank  
                            3               you  
                            4             thank  
                            5               you  

[543926 rows x 4 columns]

In [5]:
CORPUS.to_csv('CORPUS.csv', sep='|')